In [ ]:
import os
os.environ["GROQ_API_KEY"] = "add-your-api-key-here"

## Import libraries

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq.chat_models import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## Step 1a - Indexing (Document Ingestion)

In [4]:
video_id = "QmOF0crdyRU" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

- Welcome to the Huberman Lab Podcast, where we discuss science
and science-based tools for everyday life. [energetic music] I'm Andrew Huberman, and I'm a professor of
neurobiology and ophthalmology at Stanford School of Medicine. Today, we are going to
talk all about dopamine and what drives you to do
the things that you do. We're going to talk about
motivation and desire and craving, but also how dopamine
relates to satisfaction and our feelings of wellbeing. And of course, any
discussion about dopamine has to include a discussion
about the potential for dopamine induced addiction. Indeed, dopamine lies at
the heart of addiction to all things. But today we are mainly going to focus on, how what we do and how we do it, and how we conceptualize those things leads to changes in this amazing molecule in our brain and bodies
that we call dopamine. I'm going to teach you what
dopamine is and what it is not. There are a lot of myths
about the molecule dopamine. We often hear about
so-calle

In [5]:
transcript_list

[{'text': '- Welcome to the Huberman Lab Podcast,',
  'start': 0.053,
  'duration': 2.247},
 {'text': 'where we discuss science\nand science-based tools',
  'start': 2.3,
  'duration': 2.63},
 {'text': 'for everyday life.', 'start': 4.93, 'duration': 1.201},
 {'text': '[energetic music]', 'start': 6.131, 'duration': 3.399},
 {'text': "I'm Andrew Huberman,", 'start': 9.53, 'duration': 0.97},
 {'text': "and I'm a professor of\nneurobiology and ophthalmology",
  'start': 10.5,
  'duration': 2.85},
 {'text': 'at Stanford School of Medicine.', 'start': 13.35, 'duration': 2.07},
 {'text': 'Today, we are going to\ntalk all about dopamine',
  'start': 15.42,
  'duration': 2.21},
 {'text': 'and what drives you to do\nthe things that you do.',
  'start': 17.63,
  'duration': 3.56},
 {'text': "We're going to talk about\nmotivation and desire and craving,",
  'start': 21.19,
  'duration': 4.34},
 {'text': 'but also how dopamine\nrelates to satisfaction',
  'start': 25.53,
  'duration': 3.68},
 {'t

## Step 1b - Indexing (Text Splitting)

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [7]:
len(chunks)

168

In [8]:
chunks[100]

Document(metadata={}, page_content="didn't engage in the activity would cause the release of\ndopamine to a substantial degree. They do cause the release of dopamine to a substantial degree. And over time that will\ndeplete your dopamine. So energy drinks, pre-workout drinks, drugs of various kinds\nthat people take to study and pay attention. We talked about some of\nthese for the ADHD episode, things like Adderall, Ritalin,\narmodafinil or modafinil taken repeatedly over time will reduce the level of satisfaction and joy that you get from the activities you engage in while under the\ninfluence of those compounds. I'm not trying to demonize those compounds for their clinical use. What I'm saying is taking stimulants and then engaging in activities that you would like to\ncontinue to feel pleasureful is undercutting the process. And inevitably might not happen tomorrow, might not happen in a month, but inevitably you will have\nchallenges with motivation and drive related to those acti

## Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vector_store = FAISS.from_documents(chunks, embeddings)

c:\Users\Omkar\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
vector_store.index_to_docstore_id

{0: '9ae24141-367f-4944-b83d-d7d7de82e083',
 1: 'e90b21f0-2a15-42b2-b2c4-ba1165043693',
 2: '8c14b778-da32-444f-88ed-936cc0a6c301',
 3: 'f148c658-3b12-42c7-986f-95def56f4fbd',
 4: '9ea81635-4e7e-44d5-a91f-efee551d1686',
 5: '2abc5dc6-6d07-45eb-947b-6e7c5f19da9c',
 6: '5890afae-2575-44ae-8ad5-885ff2a37fca',
 7: 'e4c0e3ed-468a-455f-96df-bbae21b1df7b',
 8: '683eaec4-a24a-46f4-916b-59bedd46fbd2',
 9: 'dc5d57b8-4479-4428-9778-97d84a20c8c9',
 10: '2db77bc0-ca6f-49ba-b946-074e459d3634',
 11: 'a5e28ef3-1b3e-4ce3-ae87-297cfa7b8706',
 12: '9ad6ca9c-f142-438e-a8aa-5b19e85b468a',
 13: '225543a6-d541-4f34-a964-75bb6b2fd9d8',
 14: '15479e8f-d14c-44a8-9192-618c57b0e882',
 15: 'b9c0b401-4130-44b0-944d-664f9d0b127d',
 16: 'a97f2c60-e5aa-435a-b99d-641690ea6f6a',
 17: '699b27a7-e3b4-4fb5-86bf-4f63e6b2fafe',
 18: 'c3ab6346-8a26-47ea-9d00-79f6fc6ac411',
 19: 'f030bd45-e7e4-4587-95a2-bf9839577d6a',
 20: 'dbc5bf98-0ca0-4744-bd69-d54332340ee9',
 21: '88364869-6a42-4af4-843f-0e295b7f7893',
 22: '54aa1d8b-6283-

In [11]:
vector_store.get_by_ids(['93e8cd17-9e02-44f5-9fe5-16b9cb0f9792'])

[]

## Step 2 - Retrieval

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [13]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000296E8613F10>, search_kwargs={'k': 4})

In [30]:
retriever.invoke('What is Dopamine')

[Document(id='a17e30e4-50a0-440d-a4a6-78d410ea332f', metadata={}, page_content="get into even more detail. We'll put a link to that\nin the caption on YouTube. Right now, I want to share\nwith you two anecdotes, one from my own life, and one from some fairly recent history that illustrate some of the\ncore biology of dopamine and how profoundly it\ncan shape our experience. The first one is a really\ntragic situation that occurred. This was in the 80s, there was a outbreak of what\nlooked like Parkinsonian symptoms in a young population. So many of you heard\nof Parkinson's disease. Parkinson's disease is a disease\nin which people initially start to quake, can't"),
 Document(id='88364869-6a42-4af4-843f-0e295b7f7893', metadata={}, page_content="dopamine influences motivation, drive and craving. It involves structures that\nsome of you may have heard of before. Things like nucleus accumbens\nand the prefrontal cortex. This is the pathway that\nreally gets disrupted in addictions where i

## Step 3 - Augmentation

In [52]:
llm = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.2)

In [53]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [54]:
question          = "is the topic of Dopamine discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [55]:
retrieved_docs

[Document(id='a17e30e4-50a0-440d-a4a6-78d410ea332f', metadata={}, page_content="get into even more detail. We'll put a link to that\nin the caption on YouTube. Right now, I want to share\nwith you two anecdotes, one from my own life, and one from some fairly recent history that illustrate some of the\ncore biology of dopamine and how profoundly it\ncan shape our experience. The first one is a really\ntragic situation that occurred. This was in the 80s, there was a outbreak of what\nlooked like Parkinsonian symptoms in a young population. So many of you heard\nof Parkinson's disease. Parkinson's disease is a disease\nin which people initially start to quake, can't"),
 Document(id='9ae24141-367f-4944-b83d-d7d7de82e083', metadata={}, page_content="- Welcome to the Huberman Lab Podcast, where we discuss science\nand science-based tools for everyday life. [energetic music] I'm Andrew Huberman, and I'm a professor of\nneurobiology and ophthalmology at Stanford School of Medicine. Today, we a

In [56]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

'get into even more detail. We\'ll put a link to that\nin the caption on YouTube. Right now, I want to share\nwith you two anecdotes, one from my own life, and one from some fairly recent history that illustrate some of the\ncore biology of dopamine and how profoundly it\ncan shape our experience. The first one is a really\ntragic situation that occurred. This was in the 80s, there was a outbreak of what\nlooked like Parkinsonian symptoms in a young population. So many of you heard\nof Parkinson\'s disease. Parkinson\'s disease is a disease\nin which people initially start to quake, can\'t\n\n- Welcome to the Huberman Lab Podcast, where we discuss science\nand science-based tools for everyday life. [energetic music] I\'m Andrew Huberman, and I\'m a professor of\nneurobiology and ophthalmology at Stanford School of Medicine. Today, we are going to\ntalk all about dopamine and what drives you to do\nthe things that you do. We\'re going to talk about\nmotivation and desire and craving, bu

In [57]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [58]:
final_prompt

StringPromptValue(text='\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don\'t know.\n\n      get into even more detail. We\'ll put a link to that\nin the caption on YouTube. Right now, I want to share\nwith you two anecdotes, one from my own life, and one from some fairly recent history that illustrate some of the\ncore biology of dopamine and how profoundly it\ncan shape our experience. The first one is a really\ntragic situation that occurred. This was in the 80s, there was a outbreak of what\nlooked like Parkinsonian symptoms in a young population. So many of you heard\nof Parkinson\'s disease. Parkinson\'s disease is a disease\nin which people initially start to quake, can\'t\n\n- Welcome to the Huberman Lab Podcast, where we discuss science\nand science-based tools for everyday life. [energetic music] I\'m Andrew Huberman, and I\'m a professor of\nneurobiology and ophthalmology at St

## Step 4 - Generation

In [59]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of Dopamine is discussed in this video. 

The discussion includes:
- An introduction to dopamine and its role in motivation, desire, craving, satisfaction, and wellbeing.
- The myths about dopamine, specifically about "dopamine hits".
- The biology and psychology of dopamine.
- Neural circuits related to dopamine.
- "Dopamine schedules" - how different substances (food, drugs, caffeine, etc.) can change baseline levels of dopamine and affect how much dopamine we can experience.
- The relationship between dopamine and addiction.
- The "pleasure-pain balance" and how it relates to seeking and indulging in pleasurable activities.

The host, Andrew Huberman, also mentions two books about dopamine: "Dopamine Nation" by Dr. Anna Lembke and "The Molecule of More".


## Building a Chain

In [60]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [61]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [62]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [63]:
parallel_chain.invoke('who is Demis')

{'context': "nowadays to see people texting and doing selfies and\ncommunicating in various ways, listening to podcasts, listening to music, doing all sorts of\nthings while they engage in other activities or going to dinner and texting other people or making plans, sharing information. That's all wonderful, it gives depth and\nrichness and color to life, but it isn't just about\nour distracted nature when we're engaging with the phone, it's also a way of layering in dopamine. And it's no surprise\nthat levels of depression and lack of motivation are\nreally on the increase. Everything that we've\ntalked about until now sets up an explanation or interpretation of why interacting with digital technology can potentially lead to disruptions or lowering in baseline\nlevels of dopamine. I can use a personal example for this. I happen to really enjoy working out, I've always really enjoyed it. But in recent years I noticed that, if I was bringing my phone to my workouts, then not only was I 

In [64]:
parser = StrOutputParser()

In [65]:
main_chain = parallel_chain | prompt | llm | parser

In [66]:
main_chain.invoke('Can you summarize the video')

"Here's a summary based on the provided transcript:\n\nThe video is about dopamine, a molecule in the brain that drives motivation, desire, and craving. The host, Andrew Huberman, aims to discuss the biology of dopamine, its role in shaping our experiences, and how it relates to satisfaction, wellbeing, and addiction. He plans to share personal anecdotes and historical examples to illustrate the core biology of dopamine. \n\nSpecifically, he wants to cover what dopamine is and what it's not, debunking common myths, and explain how dopamine levels change based on our experiences and baseline levels. He also wants to discuss how repeated engagement in enjoyable activities can lead to increased thresholds for enjoyment and how understanding dopamine kinetics can help people use dopamine-enhancing strategies effectively. \n\nNo detailed summary of the whole video can be provided as the transcript seems to be truncated. However, it appears that the host will delve deeper into these topics a